In [3]:
# import all necessary variables
import numpy as np
import cv2
import glob

# annoying (fastidiosi) details of the images because i used erode and dilate method
# video 1: label in the back part of the object
# video 2: legs of the animal of the same color as the glass
# video 3: label in the back part of the object
# video 4: some pixel into the object with rgb in the range of the background

# To see that the range of the yellow color for each video is correct
# we can see the images into the glass folder.
# Then i used erode and dilate methods to delete the label, the shade of the object etc

# because i found the yellow glass and then i subtract it from the image to find
# the object in a good manner, in some image, in the glass borders there is a 1-2 black pixel
# that they appear as if they are in the main object.
# I left these pixel because using the projection method, they not appear in all frames and for
# for this, they don't appear in the ply file.

# bitwise and: when a mask is provided the AND operation is performed between the first
# and second image arrays only where the corresponding mask values do not equal zero

# erode: it needs two inputs, one is our original image, second one is called structuring element
# or kernel which decides the nature of operation.
# A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the
# kernel is 1, otherwise it is eroded (made to zero)

# dilate: opposite of the erosion. In fact a pixel element in the original image is ‘1’ if at least one pixel under the kernel is ‘1’

# variable to store which video we must elaborate, we must insert only numbers 1,2,3 or 4
videoToElaborate = 4

# get list of frames of the chosen video
listImages = glob.glob("framesVideo" + str(videoToElaborate) + "/uncalibrated/*.jpg")

# for each image
for image in listImages:

    # open the actual image in color mode
    inputImage = cv2.imread(image, cv2.IMREAD_COLOR)

    # open the actual image in color mode to elaborate in the last part of this algorithm
    imageToElaborate = cv2.imread(image, cv2.IMREAD_COLOR)

    # dimension of the object plus a small piece of the glass
    # to have less types of blue in the background
    imageObjectGlass = inputImage[:, int(1920 / 6):int(1920 / 2) + int(1920 / 30), :]

    # image with the glass and the small bottom part of the object
    # to divide then better the object and the glass
    imageGlassBottomPartObject = imageObjectGlass[:, 509:736, :]

    # white background and black object
    mask2 = cv2.inRange(imageObjectGlass, (40, 0, 0), (173, 120, 96))

    # blue background and black object
    target = cv2.bitwise_and(imageObjectGlass, imageObjectGlass, mask=mask2)

    # create a fake mask to choose the definitive one in the if cycle
    mask3 = cv2.inRange(imageGlassBottomPartObject, (90, 170, 170), (170, 255, 255))

    # this case works
    if videoToElaborate == 1:

        # i know that i lost a very small bottom part of the object and it is due to the erode and dilate
        # methods that i used to delete the label from the object loosing quality but so i obtain a good 3d model
        # with all inner points.

        # i create the two masks for the yellow color of the glass
        mask3 = cv2.inRange(imageGlassBottomPartObject, (0, 165, 160), (110, 255, 255))
        mask4 = cv2.inRange(imageGlassBottomPartObject, (160, 165, 160), (190, 255, 255))

        # i create the definitive mask using the previous two
        mask3 = cv2.bitwise_or(mask3, mask4)

        # yellow glass and black background
        target1 = cv2.bitwise_and(imageGlassBottomPartObject, imageGlassBottomPartObject, mask=mask3)

        # create a kernel matrix of 3x3
        kernel = np.ones((3, 3), np.uint8)

        # erosion and dilatation to divide better the glass and the bottom part of the object
        target1 = cv2.erode(target1, kernel, iterations=10)
        target1 = cv2.dilate(target1, kernel, iterations=25)
        target1 = cv2.erode(target1, kernel, iterations=3)

        # erosion of the object plus glass to delete the label of the back part of the object
        target = cv2.erode(target, kernel, iterations=10)

    elif videoToElaborate == 2:

        # I know very well that using erosion and dilation the image loses quality.
        # Otherwise if I didn't I couldn't highlight the legs of the animal
        # in black at 100% and there were empty holes in the middle.
        # Then another important thing is the black border of the glass, i can delete it using dilate method
        # but i loss precision in the legs. And if i delete that color from the range, some pixel of the legs
        # of the animal are lost. And this border is deleted from the 3d model thanks to the algorithm of projection

        # i create a mask for the yellow color of the glass
        mask3 = cv2.inRange(imageGlassBottomPartObject, (0, 185, 160), (110, 255, 255))

        # create the kernel as 3x3
        kernel = np.ones((3, 3), np.uint8)

        # yellow glass and black background
        target1 = cv2.bitwise_and(imageGlassBottomPartObject, imageGlassBottomPartObject, mask=mask3)

        # erosion and dilatation to divide better the glass and the bottom part of the object
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=12)
        target1 = cv2.dilate(target1, kernel, iterations=6)

    elif videoToElaborate == 3:

        # i know that i lost a very small bottom part of the object and it is due to the erode and dilate
        # methods that i used to delete the label in front of the object loosing quality but so i obtain a good 3d model.
        # One defect of this algorithm is that the label in the back part of the object is of the same color of the background
        # and to delete it, we may use erode method losing quality but i dont't do it to have more quality
        # but there is a small hole in the 3d model but for the target of this project that's okay.

        # i create a mask for the yellow color of the glass
        mask3 = cv2.inRange(imageGlassBottomPartObject, (0, 140, 150), (110, 255, 255))

        # yellow glass and black background
        target1 = cv2.bitwise_and(imageGlassBottomPartObject, imageGlassBottomPartObject, mask=mask3)

        # create the kernel as 3x3
        kernel = np.ones((3, 3), np.uint8)

        # erode and dilate methods to divide better the glass and the bottom part of the object
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=5)
        target1 = cv2.erode(target1, kernel, iterations=5)

        # erosion of the object plus glass to delete the label of the front part of the object
        target = cv2.erode(target, kernel, iterations=10)
        target = cv2.dilate(target, kernel, iterations=8)

        # i didn't the erosion for the label in the back part of the object otherwise
        # we lose much precision of the object and for this we have only a small hole in the 3d model

    elif videoToElaborate == 4:

        # i know that i lost a very small bottom part of the object and it is due to the erode and dilate methods.
        # In fact into the object there are some pixel that are in the rgb range of the background
        # and to include them into the object i must used erode and dilate methods

        # i create a mask for the yellow color of the glass
        mask3 = cv2.inRange(imageGlassBottomPartObject, (0, 155, 155), (110, 255, 255))

        # yellow glass and black background
        target1 = cv2.bitwise_and(imageGlassBottomPartObject, imageGlassBottomPartObject, mask=mask3)

        # create the kernel as 3x3
        kernel = np.ones((3, 3), np.uint8)

        # erode and dilate methods to divide better the glass and the bottom part of the object
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=3)
        target1 = cv2.dilate(target1, kernel, iterations=8)
        target1 = cv2.erode(target1, kernel, iterations=7)

        # erode and dilate methods to get better the whole object because there are some shade
        # with rgb code not divisible from the yellow glass
        target = cv2.erode(target, kernel, iterations=4)
        target = cv2.dilate(target, kernel, iterations=3)

    # save in the glass folder the image with yellow glass and black background
    glass = image.replace("uncalibrated", "glass")
    cv2.imwrite(glass, target1)

    # save in the object folder the image with both object and glass
    objectWithGlass = image.replace("uncalibrated", "object")
    cv2.imwrite(objectWithGlass, target)

    # for each y of the original image
    for y in range(0, 1080):

        # for x in the small part of the object
        for x in range(int(1920 / 6), int(1920 / 2) + int(1920 / 30)):

            # if i am in the small bottom part of the object and in the glass coordinates
            if int(1920 / 6) + 509 <= x <= int(1920 / 2) + int(1920 / 30):

                # if i am not in the background coordinates
                if target[y][x - 320][0] == 0 and target[y][x - 320][1] == 0 and target[y][x - 320][2] == 0:

                    # if i am not in the glass coordinates
                    if target1[y][x - (int(1920 / 6) + 509)][0] == 0 and target1[y][x - (int(1920 / 6) + 509)][1] == 0 and target1[y][x - (int(1920 / 6) + 509)][2] == 0:
                        # it means that i am in the object coordinates

                        # so we set the object in black color in the original image
                        imageToElaborate[y, x] = (0, 0, 0)

            # if i am in the object and not in the glass coordinates
            elif int(1920 / 6) <= x <= int(1920 / 6) + 509:

                # if i am in the object coordinates
                if target[y][x - 320][0] == 0 and target[y][x - 320][1] == 0 and target[y][x - 320][2] == 0:
                    # we set the object in black color in the original image
                    imageToElaborate[y, x] = (0, 0, 0)

    # save the image with background removed and black object in its folder
    backgroundRemovedImage = image.replace("uncalibrated", "backgroundremoved")
    cv2.imwrite(backgroundRemovedImage, imageToElaborate)
